In [ ]:
# !pip install -e /user/rea3/AP-ReA/jupyter-ML/pkgs/objFuncs
# !pip install -e /user/rea3/AP-ReA/jupyter-ML/pkgs/pyBO

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from phantasy import caget

In [ ]:
import objFuncs
from objFuncs import residuals
from objFuncs.util import plot_obj_history

In [ ]:
objFuncs._global_machineIO._test = True
objFuncs._global_machineIO._fetch_data_time_span = 2.05
objFuncs._global_machineIO._ensure_set_timewait_after_ramp = 0.25
objFuncs._global_machineIO._verbose = True
objFuncs._global_machineIO.view()

In [ ]:
ion = caget("REA_EXP:ELMT")
Q = int(caget("REA_EXP:Q"))
A = int(caget("REA_EXP:A"))
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print(ion, 'A/Q=',AQ)

In [ ]:
now0 = datetime.datetime.now()
now0str = str(now0)[:str(now0).rfind(':')].replace(' ','_').replace(':','').replace('-','')
fname = '[REA][LSQ]'+now0str+'_'+ion+'_Qcentering'
fname

# Decision parameters and range

In [ ]:
decision_CSETs = ['REA_BTS31:DCH_D1393:I_CSET',
#                   'REA_BTS31:DCV_D1393:I_CSET',
                  'REA_BTS31:DCH_D1423:I_CSET',
#                   'REA_BTS31:DCV_D1423:I_CSET',
                  ]
STC = 50.
STC1=STC
STC2=STC

decision_min = [-STC1*AQ,-STC2*AQ]
decision_max = [ STC1*AQ, STC2*AQ]
decision_tol = [       2,       2]

# decision_min = -STC*AQ
# decision_max = -decision_min

print(decision_min)
print(decision_max)

# Conditions over which the variation of objective will be minimized

##### example:
The following:

    conditional_SETs = {
        'Q1:I_CSET':[100,120,100],
        'Q2:I_CSET':[100,100,120],
    }
    
means that we will evaluate objectives, (at given decision parameters, x) at 3 different conditions:

    Obj1(x) at Q1,Q2=(100,100),
    Obj2(x) at Q1,Q2=(120,100), 
    Obj3(x) at Q1,Q2=(100,120),
    
Then find root of "Obj2(x)-Obj1(x)" and "Obj3(x)-Obj1(x)" by deciding the decision paratemers under linear response assumption. In other words, we minimize variation of objectives over different conditions.

In [ ]:
conditional_SETs = {
    'REA_BTS31:Q_D1431:I_CSET':[80,100],
    'REA_BTS31:Q_D1435:I_CSET':[80,100],
}

n_condition = None
for k,v in conditional_SETs.items():
    if not n_condition:
        n_condition = len(v) 
    else:
        assert len(v) == n_condition

In [ ]:
#  the values objective_goal is not meaningful unless "var_obj_weight_fraction < 1" 
#  "var_obj_weight_fraction" is used in the "residualObjMultiConditionalVar" class instantiation
#  "var_obj_weight_fraction = 1" will optimize only the variation of objective values over different conditions
objective_goal = { 
        'SOMEWHERE:VIWER:XPOS_RD' : 0.0,     
#         'SOMEWHERE:VIWER:YPOS_RD' : 0.0, 
        }
#  objective_norm and objective_weight is important as they define the relative importance of each objective
objective_norm = { 
        'SOMEWHERE:VIWER:XPOS_RD' : 1.0,     #(mm)
#         'SOMEWHERE:VIWER:YPOS_RD' : 1.0,  
        }
objective_weight = { 
        'SOMEWHERE:VIWER:XPOS_RD' : 1.0,     #(mm)
#         'SOMEWHERE:VIWER:YPOS_RD' : 1.0,  
        }

In [ ]:
res = residuals.residualObjMultiConditionalVar(  
    decision_CSETs = decision_CSETs,
    decision_min   = decision_min,
    decision_max   = decision_max,
    decision_tol   = decision_tol,
    objective_goal = objective_goal,
    objective_norm = objective_norm,
    objective_weight = objective_weight,
    conditional_SETs = conditional_SETs,
    var_obj_weight_fraction = 1,
    )

In [ ]:
plot_decision = plot_obj_history(
            res.history['decision_CSETs'],
            )
plot_RDs = plot_obj_history(
            res.history['condition0']['objective_RDs'],
            )
plot_objs = plot_obj_history(
            res.history['condition0']['objectives'],
            )
callbacks = [plot_decision,plot_RDs,plot_objs]

In [ ]:
# this will allow manual input of Viewer reading
_manual_fetcher = objFuncs.construct_machineIO.construct_manual_fetch_data(objective_goal.keys())
objFuncs._global_machineIO.fetch_data = _manual_fetcher

# Run optimization
except: n_read = (n_decision+2) $\times$ n_condition

In [ ]:
print( (len(decision_CSETs)+2)*n_condition )

In [ ]:
# Whenver prompt is active, just read and type in
result = res.lsq_linear(jac_use3points=False,callbacks=callbacks)
for f in callbacks:
    f.close()

In [ ]:
print("== Final decisions ==")
for pv,x in zip(decision_CSETs,result.x):
    print(" ",pv,x)

In [ ]:
res.history['objectives_var']["values"]

In [ ]:
res.history['objectives_var']['names']

In [ ]:
for n,v in zip(res.history['jacobian_var']['names'], res.history['jacobian_var']['values'][0]):
    print(n,v)